In [4]:
# set required info

# TEST_SET_KEY = 'GTSQA-325'
tests = "GTSQA-323, GTSQA-325, GTSQA-317"
test_set_keys = [key.strip().upper() for key in tests.replace(",", " ").split()]

# VERSIONS = ["Kuiper2.0 Rel1"]


In [5]:
# Initialize script
from atlassian import Jira, Xray

JIRA_URL = "https://jira.analog.com"
API_TOKEN = "put your api key here"
PROJECT_KEY = "GTSQA"

jira = Jira(
    url=JIRA_URL,
    token=API_TOKEN
)

xray = Xray(
    url=JIRA_URL,
    token=API_TOKEN
)



In [6]:
for test_set_key in test_set_keys:
    print("="*50)
    print(f"Test Sets: {test_set_key}")

    try:
        # Get info
        test_set_summary = jira.issue_field_value(test_set_key, "summary")
        test_set_components = jira.issue_field_value(test_set_key, "components")
        
        test_set = xray.get_tests_with_test_set(test_set_key)
        if len(test_set) == 0:
            print(f"No tests found in {test_set_key}, skipping.\n")
            continue

        print(f"Summary: {test_set_summary}")
        print(f"Components: {test_set_components}")
        print(f"Tests: {[test['key'] for test in test_set]}")

        # Parse Summary
        summary = test_set_summary.split(" test")[0] if " test" in test_set_summary else test_set_summary

        # Find matching versions
        version_found = []
        for version in jira.get_project_versions(PROJECT_KEY):
            if version["name"] in VERSIONS:
                version_found.append(version["name"])

        # Build Test Execution
        issue_dict = {
            "project": {"key": PROJECT_KEY},
            "summary": f"Test {summary}",
            "issuetype": {"name": "Test Execution"},
            "description": f"Test Execution for {test_set_key} {test_set_summary}",
            "assignee": {"name": jira.myself()["name"]},
            # "reporter": {"name": "jiraUserName"},
            "versions": [{'name': version} for version in version_found],
            "components": [{"name": comp["name"]} for comp in test_set_components],
            "labels": ["Kuiper2.0-2024_r2-pre-RC0"] 
        }

        # Create and Link
        print(issue_dict)
        test_exec = jira.issue_create(issue_dict)
        test_exec_key = test_exec["key"]
        print(f"Created Test Execution: {test_exec_key}")

        xray.update_test_execution(test_exec_key, add=[test["key"] for test in test_set])
        print(f"Linked tests to {test_exec_key}")

    except Exception as e:
        print(f"Error on {test_set_key}: {e}")


Test Sets: GTSQA-323
Summary: rpi4b-cn0511 tests
Components: [{'self': 'https://jira.analog.com/rest/api/2/component/68232', 'id': '68232', 'name': 'rpi4b'}]
Tests: ['GTSQA-165', 'GTSQA-173', 'GTSQA-182', 'GTSQA-183', 'GTSQA-185', 'GTSQA-184', 'GTSQA-187', 'GTSQA-186', 'GTSQA-175', 'GTSQA-324']
{'project': {'key': 'GTSQA'}, 'summary': 'Test rpi4b-cn0511', 'issuetype': {'name': 'Test Execution'}, 'description': 'Test Execution for GTSQA-323 rpi4b-cn0511 tests', 'assignee': {'name': 'KPaller'}, 'versions': [{'name': 'Kuiper2.0 Rel1'}], 'components': [{'name': 'rpi4b'}]}
Created Test Execution: GTSQA-570
Linked tests to GTSQA-570
Test Sets: GTSQA-325
Summary: rpi4b-cn0575 tests
Components: [{'self': 'https://jira.analog.com/rest/api/2/component/68232', 'id': '68232', 'name': 'rpi4b'}]
Tests: ['GTSQA-165', 'GTSQA-173', 'GTSQA-182', 'GTSQA-183', 'GTSQA-185', 'GTSQA-184', 'GTSQA-187', 'GTSQA-186', 'GTSQA-175', 'GTSQA-326']
{'project': {'key': 'GTSQA'}, 'summary': 'Test rpi4b-cn0575', 'issuet